## Initialize Session
Run the following pipeline from a subdirectory of the XMC run directory (where all the deconvolution files are).
(The thousands of files in the main run directory slows things down.)

In [1]:
%run ../../diagnostic_init.py
outroot = './'
# Set up plotting to notebook
bplt.output_notebook()

# edit blobcols depending on specific model for this run
print blobcols
blobcols = ['blob_kT','blob_em','blob_nH','blob_sigma']
#blobcols.append()

# if level 1 cleaning already has been done, read in cleaned blobs, changing iteration numbers. 
# Skip the xd.clean below
dfall = pd.read_table('deconvolution_merged_iter300-1068.txt',index_col=0,sep=r'\s+',engine='python')

Loading BokehJS ...

['blob_nH', 'blob_kT', 'blob_Ne', 'blob_Mg', 'blob_Si', 'blob_S', 'blob_Fe', 'blob_tau', 'blob_phi', 'blob_psi', 'blob_sigma', 'blob_numberdensity']


# Check Run Progress and Filter Data
Skip if already done and merged deconvolution file is being used.

In [ ]:
sf=xplt.chi2(runpath='../',outfile='notebook')
# all chi2=0

Set minimum converged iteration, based on chi2 plot.
Change distance depending on the object.


In [2]:
itermin = 300
# determine converged chi2/dof
#sf = xw.filterblobs(sf,'iteration',minvals=itermin)
#print xw.weighted_median(sf['redchi2']) 

# remove iterations before convergence, add new derived columns, and save merged deconvolution file
#dfall = xd.clean(runpath='../',itmin=itermin,distance=3.0)

# Get Basic Blob Info
# print columns
print dfall.columns
# print number of blobs
len(dfall.index)
# print min, max of each parameter
for p in dfall.columns:
    print p,min(dfall[p]),max(dfall[p])

Index([u'blob_nH', u'blob_kT', u'blob_lnsigma', u'blob_norm', u'iteration',
       u'blob_sigma', u'blob_em', u'blob_volume', u'blob_numberdensity',
       u'blob_mass'],
      dtype='object')
blob_nH 0.2020577 0.4560139
blob_kT 0.1205994 0.1719507
blob_lnsigma 1.792241 4.999315
blob_norm 0.02537303 0.288858
iteration 300 1068
blob_sigma 6.00288988036 148.3115309
blob_em 2.73232343737e+57 3.11060004845e+58
blob_volume 3.07964316603e+53 4.64455405888e+57
blob_numberdensity 1.25094391379 228.466080995
blob_mass 6.73345123204e+31 2.00729207241e+34


# Initial Inspection

In [ ]:
# Plot blob histograms
hfigs=xplt.histogram_grid(dfall[blobcols],outfile='notebook',ncols=2)
whfigs=xplt.histogram_grid(dfall[blobcols],outfile='notebook',ncols=2,weights=dfall['blob_em'])

In [ ]:
# Plot traceplots
# if Datashader not available, set agg=None. Note this will be MUCH slower.
tfigs = xplt.traceplots(dfall[blobcols],agg='dscount')

# Remove high EM - low kT blobs
Use the histograms and scatter plots to determine good guesses for an EM or temperature threshold.

Then create EM maps signficance maps for the 'bad' blobs to see if the emission from those blobs was significant.

In [3]:
# Scatter plots with linked brushing enabled.
interesting_cols = ['blob_kT','blob_em','blob_nH']
tfigs2 = xplt.traceplots(dfall[interesting_cols],agg='dscount',sampling=1000,outfile='notebook')
#sfigs = xplt.scatter(dfall,'blob_em','blob_kT',outfile='notebook',ylog='auto',agg=None,size=10)

Loading BokehJS ...

In [ ]:
# based on plots, set EM and/or kT threshold for 'bad' blobs
emthresh = 2e59
kTthresh = 0.12

# Filter blobs based on EM and/or kT
# change parameter name and limits as needed
dfbad = xw.filterblobs(dfall,'blob_em',minvals=emthresh)
dfbad = xw.filterblobs(dfbad,'blob_kT',maxvals=kTthresh)

# Create EM map of 'bad' blobs
# change x0, y0, imagesize, binsize as appropriate (see the start_xmc file for x0,y0)
imgs=xm.make_map(dfbad,paramname='blob_em',paramweights=None,iteration_type='total',
                 binsize=2.0,nlayers=20,imagesize=160.,withsignificance=True,nproc=4,
                 outfile=outroot+'bin2_160arcsec_bad',x0=-60.,y0=-10.,clobber=True)

# Open file in ds9
#!ds9 -multiframe 

# Once decided on proper thresholds, filter the dataframe for further data exploration
df = xw.filterblobs(dfall,'blob_em',minvals=emthresh)
df = xw.filterblobs(df,'blob_kT',maxvals=kTthresh)

# Exploring Details

In [ ]:
# Optionally plot subset of the traceplots with linked brushing enabled.
interesting_cols = ['blob_kT','blob_em','blob_nH']
tfigs2 = xplt.traceplots(df[interesting_cols],agg=None,sampling=500,outfile='notebook')

# Optionally plot subset of histograms with finer bins
hfigs2 = xplt.histogram_grid(df[interesting_cols],outfile='notebook',bins=500)
whfigs2 = xplt.histogram_grid(df[interesting_cols],outfile='notebook',bins=500,weights=df['blob_em'])

# Filter blobs based on desired parameters
# change parameter name and limits as needed
dffilt = xw.filterblobs(df,'blob_kT',minvals=0.15,maxvals=None)

